# Revisão de código 

Olá! 

Meu nome é Suelen. Estou feliz em revisar seu projeto hoje!

Quando vejo um erro pela primeira vez, apenas aponto. Deixarei você encontrá-lo e corrigi-lo sozinho. Além disso, ao longo do texto, farei algumas observações sobre melhorias no código e também farei comentários sobre suas percepções sobre o assunto. Mas se você ainda não consegue lidar com essa tarefa, darei uma dica mais precisa na próxima iteração e também alguns exemplos práticos. Estarei aberta a feedbacks e discussões sobre o tema.

Você pode encontrar meus comentários em caixas verdes, amarelas ou vermelhas como estas:


<div class="alert alert-block alert-success">
<b>Comentário: </b> <a class="tocSkip"></a>

Sucesso. Tudo está correto.
</div>

<div class="alert alert-block alert-warning">
<b>Comentário: </b> <a class="tocSkip"></a>

Observações. Algumas recomendações.
</div>

<div class="alert alert-block alert-danger">

<b>Comentário: </b> <a class="tocSkip"></a>

O bloco requer algumas correções. O trabalho não pode ser aceito com os comentários vermelhos.
</div>

Você pode me responder usando isto:

<div class="alert alert-block alert-info">
<b>Resposta do aluno</b> <a class="tocSkip"></a>
</div>



O serviço de vendas de carros usados Rusty Bargain está desenvolvendo um aplicativo para atrair novos clientes. Nesse aplicativo, você pode descobrir rapidamente o valor de mercado do seu carro. Você tem acesso a dados históricos: especificações técnicas, versões de acabamento e preços. Você precisa construir o modelo para determinar o valor. 

Rusty Bargain está interessado em:

- a qualidade da predição;
- a velocidade da predição;
- o tempo necessário para o treinamento

##### Aluno: Gabriel Chaves Veras

##### Curso: Ciência de Dados

#### Olá!! Estou com problemas nesta atividade, dei mais detalhes já no finalzinho do código. Por favor, poderia me ajudar?

<div class="alert alert-block alert-danger">
<b> Comentário geral v1: </b> <a class="tocSkip"></a>
    
    
Oi, Gabriel! Prazer em te ajudar :)  Vamos resolver isso juntos.

Você está correto ao identificar que o erro está relacionado à memória.  
Analisando seu código, notei que o dataframe usado para treinar o modelo possui **874 colunas**.  
Com esse volume de atributos, a dimensionalidade cresce bastante e o processamento fica pesado, causando falhas na execução.

Não é necessário aplicar `get_dummies` em todas as colunas. O ideal é limitar a técnica apenas às variáveis categóricas relevantes ou utilizar alternativas mais eficientes (como *OneHotEncoder* com matriz esparsa).

Exemplo para te inspirar: 

```python
from sklearn.compose import ColumnTransformer

# Identifica colunas numéricas vs. categóricas
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

# Cria pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Aplica transformação
X_train_encoded = preprocessor.fit_transform(X_train)
X_val_encoded = preprocessor.transform(X_val)
````


Estou aqui disponível para quaisquer dúvidas que possam surgir!  Boa sorte

<div class="alert alert-block alert-success">
  <b>Comentário geral v3:</b> <a class="tocSkip"></a>

Oiiii, Gabriel! Muito obrigada pelos ajustes!


Gostaria apenas de destacar um ponto importante relacionado ao uso do `get_dummies()`.

No seu dataset, algumas variáveis categóricas possuem **alta cardinalidade** (por exemplo, mais de 250 categorias diferentes em uma única coluna). Quando aplicamos `get_dummies()` diretamente nesse tipo de variável, o pandas cria **uma coluna para cada categoria**, o que resulta em uma matriz de treinamento extremamente grande e esparsa.

Isso traz alguns problemas:

* **Aumento significativo da dimensionalidade**, deixando o modelo mais pesado e lento.
* **Maior risco de overfitting**, porque o modelo aprende detalhes irrelevantes de categorias raras.
* **Uso de memória muito elevado**, especialmente quando o modelo é treinado por algoritmos implementados manualmente (como o seu SGD), podendo até causar travamentos.
* **Pouco ganho real de informação**: muitas dessas categorias aparecem poucas vezes e não contribuem de forma útil para a regressão.

Para cenários como esse, costuma ser melhor substituir get_dum por técnicas mais eficientes, como:

* **Target Encoding**
* **Frequency Encoding**
* **Agrupamento de categorias raras em “Other”**
* Ou aplicar `get_dummies` apenas nas variáveis de baixa cardinalidade.

Sua solução funciona, mas entender essas limitações ajuda a construir modelos mais escaláveis e robustos.

No mais, parabéns pelo projeto! :) 


## Preparação dos dados

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import lightgbm as lgb
import xgboost as xgb

In [2]:
# Baixando dados e transformando em dataframe:
df = pd.read_csv('/datasets/car_data.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [3]:
df.sample(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
224439,07/03/2016 13:54,1250,sedan,1997,NaN,75,golf,5000,11,petrol,volkswagen,NaN,07/03/2016 00:00,0,94032,22/03/2016 19:46
192816,03/04/2016 12:37,750,wagon,1998,manual,75,cordoba,150000,9,petrol,seat,yes,03/04/2016 00:00,0,57078,07/04/2016 14:56
254005,11/03/2016 13:52,3700,sedan,2004,manual,143,3er,125000,5,petrol,bmw,no,11/03/2016 00:00,0,63814,07/04/2016 11:18
289872,21/03/2016 16:55,3885,sedan,2000,manual,147,other,150000,10,petrol,citroen,no,21/03/2016 00:00,0,42859,06/04/2016 16:46
35545,09/03/2016 22:50,10650,bus,2010,auto,140,touran,150000,10,gasoline,volkswagen,no,09/03/2016 00:00,0,76698,11/03/2016 09:16
15978,07/03/2016 13:54,850,NaN,2000,manual,101,a3,150000,0,petrol,audi,no,07/03/2016 00:00,0,32469,06/04/2016 12:44
317632,05/04/2016 11:53,3950,bus,2007,manual,82,vivaro,125000,7,gasoline,opel,no,05/04/2016 00:00,0,13347,05/04/2016 11:53
49383,13/03/2016 14:45,700,NaN,2016,NaN,150,c_klasse,150000,3,petrol,mercedes_benz,NaN,13/03/2016 00:00,0,26409,19/03/2016 12:18
41306,16/03/2016 17:38,0,small,2003,manual,116,stilo,150000,1,gasoline,fiat,yes,16/03/2016 00:00,0,28239,21/03/2016 20:17
45102,21/03/2016 01:57,999,bus,1998,manual,141,other,150000,7,lpg,opel,no,21/03/2016 00:00,0,13353,06/04/2016 00:46


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [5]:
#Avaliando valores únicos para entender se há a necessidade de correção (escritas incorretas, caracteres especiais etc)
print(df['VehicleType'].unique())
print(df['Gearbox'].unique())
print(df['Model'].unique())
print(df['FuelType'].unique())
print(df['Brand'].unique())

[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
['manual' 'auto' nan]
['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuore' 's_max' 'a2' 'galaxy' 'c3

In [6]:
#Avaliando colunas numéricas em busca de inconsistências:
print('Coluna Price:\n',df['Price'].describe())
print('Coluna Power:\n',df['Power'].describe())
print('Coluna Mileage:\n',df['Mileage'].describe())

Coluna Price:
 count    354369.000000
mean       4416.656776
std        4514.158514
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64
Coluna Power:
 count    354369.000000
mean        110.094337
std         189.850405
min           0.000000
25%          69.000000
50%         105.000000
75%         143.000000
max       20000.000000
Name: Power, dtype: float64
Coluna Mileage:
 count    354369.000000
mean     128211.172535
std       37905.341530
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Mileage, dtype: float64


In [7]:
#Encontrado um valor fora da realidade para carros, com potências até 20000 HP. Pode ser algum erro na unidade, porém como iremos avaliar todos como HP,
#Precisamos ajustar os valores para mais próximos da realidade ou apenas desconsidera-los. Escolhi a segunda opção, pois utilizar uma média poderia mascarar os resultados.

(df.loc[df['Power'] > 1500, 'Power']) = df['Power'].median()
print('Coluna Power:\n',df['Power'].describe())

Coluna Power:
 count    354369.000000
mean        106.945560
std          67.010059
min           0.000000
25%          69.000000
50%         105.000000
75%         141.000000
max        1500.000000
Name: Power, dtype: float64


<div class="alert alert-block alert-success">
<b>Comentário: </b> <a class="tocSkip"></a>

- Os dados foram carregados e processados corretamente   
- Comentários foram adicionados explicando o racional do código 
    


In [8]:
# Convertendo dados do tipo 'object' para 'datetime', respeitando escalabilidade (caso mais colunas sejam adicionadas, basta completar a lista 'date_columns').
# Para funcionar nos algoritmos de previsão, iremos alterar a data para apenas o ano daquela data. 
date_columns = ['DateCrawled', 'DateCreated', 'LastSeen']

# Convertendo todas de uma vez e criando novo dataframe para não perder dados do original:
df_car_prices = df.copy()
for col in date_columns:
    df_car_prices[col] = pd.to_datetime(df_car_prices[col]).dt.year


In [9]:
#Definindo nossos atributos e objetivo:
features = df_car_prices.drop(['Price'], axis=1)
target = df_car_prices['Price']


In [12]:
features.dtypes

DateCrawled            int64
VehicleType           object
RegistrationYear       int64
Gearbox               object
Power                float64
Model                 object
Mileage                int64
RegistrationMonth      int64
FuelType              object
Brand                 object
NotRepaired           object
DateCreated            int64
NumberOfPictures       int64
PostalCode             int64
LastSeen               int64
dtype: object

In [13]:
features['Model'].value_counts().head(20)
len(features['Model'].unique())


251

In [10]:
# Identifica colunas numéricas vs. categóricas
numeric_cols = features.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = features.select_dtypes(include=['object', 'category']).columns

#Preenchendo valores nulos:
for col in categorical_cols:
    features[col] = features[col].fillna('Unknown')

#Criando colunas Dummy para conseguir rodar a regressão linear nas colunas categóricas. 
features_dummies = pd.get_dummies(features, drop_first=True)

In [11]:
# Verificar se há valores NaN
print("Valores NaN por coluna:")
print(features_dummies.isnull().sum().sum())

Valores NaN por coluna:
0


In [12]:
# Identifica colunas numéricas vs. categóricas
#numeric_cols = features.select_dtypes(include=['int64', 'float64']).columns
#categorical_cols = features.select_dtypes(include=['object']).columns

# Preencher valores nulos ANTES da transformação
#for col in categorical_cols:
#    features[col] = features[col].fillna('Unknown')

#for col in numeric_cols:
#    features[col] = features[col].fillna(features[col].median())

## Criar o preprocessor
#preprocessor = ColumnTransformer(
#    transformers=[
#        ('num', 'passthrough', numeric_cols),
#        ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_cols)
#    ]
#)

# Aplicar transformação
#features_encoded = preprocessor.fit_transform(features)
#print(f"Forma final: {features_encoded.shape}")

In [11]:
# Identifica colunas numéricas vs. categóricas
numeric_cols = features.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = features.select_dtypes(include=['object', 'category']).columns

# Preenchendo valores nulos somente nas categóricas
features[categorical_cols] = features[categorical_cols].fillna('Unknown')

# Aplicando get_dummies somente nas variáveis categóricas
categorical_dummies = pd.get_dummies(features[categorical_cols], drop_first=True)

# Concatenando numéricas + dummies
features_dummies = pd.concat([features[numeric_cols], categorical_dummies], axis=1)
features_dummies

,DateCrawled,RegistrationYear,Power,Mileage,RegistrationMonth,DateCreated,NumberOfPictures,PostalCode,LastSeen,VehicleType_bus,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
0,2016,1993,0.0,150000,0,2016,0,70435,2016,0,...,0,0,0,0,0,0,1,0,0,0
1,2016,2011,190.0,125000,5,2016,0,66954,2016,0,...,0,0,0,0,0,0,0,0,0,1
2,2016,2004,163.0,125000,8,2016,0,90480,2016,0,...,0,0,0,0,0,0,0,0,0,0
3,2016,2001,75.0,150000,6,2016,0,91074,2016,0,...,0,0,0,0,0,0,1,0,1,0
4,2016,2008,69.0,90000,7,2016,0,60437,2016,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354364,2016,2005,0.0,150000,7,2016,0,2694,2016,0,...,0,0,0,0,0,0,0,0,0,1
354365,2016,2005,0.0,20000,1,2016,0,39576,2016,0,...,0,1,0,0,0,0,0,0,0,0
354366,2016,2000,101.0,125000,3,2016,0,26135,2016,0,...,1,0,0,0,0,0,0,0,1,0
354367,2016,1996,102.0,150000,3,2016,0,87439,2016,1,...,0,0,0,0,0,0,1,0,1,0


In [ ]:
O treinamento dos modelos ainda continua usando o dataframe com as 317 colunas. Precisamos ajustar para que i orocessamento das fearuyres esteja correto.

Dica: 

```python
# Identifica colunas numéricas vs. categóricas
numeric_cols = features.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = features.select_dtypes(include=['object', 'category']).columns

# Preenchendo valores nulos somente nas categóricas
features[categorical_cols] = features[categorical_cols].fillna('Unknown')

# Aplicando get_dummies somente nas variáveis categóricas
categorical_dummies = pd.get_dummies(features[categorical_cols], drop_first=True)

# Concatenando numéricas + dummies
features_dummies = pd.concat([features[numeric_cols], categorical_dummies], axis=1)

```

<div class="alert alert-block alert-danger">
<b> Comentário: </b> <a class="tocSkip"></a>
    
    
Recomendo modificar a parte do código na célula acima, especificamente na parte do `pd.get_dummies`. Tente processar de forma a não aumentar em grande escala o número de colunas. Por exemplo, utilize o get_dumies apenas em algumas colunas do dataframe ao invés do dataframe todo. 


## Treinamento do modelo

In [14]:
#Separando conjuntos de treinamento e validação para features e target:
features_train, features_valid, target_train, target_valid = train_test_split(features_dummies, target, test_size=0.25, random_state=12345)

- ##### Modelo de Regressão Linear com gradiente descendente estocástico:

Abaixo, tentei utilizar o código para gradiente estocástico de regressão linear, mas não consegui. Fiz ajustes nos parâmetros, revi equações e nada.
O que poderia ser feito para resolver? (Atualmente as células estão em markdown para não gerarem muita complexidade na hora de rodar o programa)

%%time
class SGDLinearRegression:
    def __init__(self, step_size, epochs, batch_size, reg_weight):
        self.step_size = step_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.reg_weight = reg_weight

    def fit(self, train_features, train_target):
        X = np.concatenate(
            (np.ones((train_features.shape[0], 1)), train_features), axis=1
        )
        y = train_target
        w = np.zeros(X.shape[1])

        for _ in range(self.epochs):
            batches_count = X.shape[0] // self.batch_size
            for i in range(batches_count):
                begin = i * self.batch_size
                end = (i + 1) * self.batch_size
                X_batch = X[begin:end, :]
                y_batch = y[begin:end]

                gradient = (
                    2
                    * X_batch.T.dot(X_batch.dot(w) - y_batch)
                    / X_batch.shape[0]
                )
								
                reg = 2 * w.copy()
                reg[0] = 0
                gradient += self.reg_weight * reg
                
                w -= self.step_size * gradient

        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

%%time
model2 = SGDLinearRegression(0.00001, 10, 1000, 0.1)
model2.fit(features_train, target_train)
predicted_valid2 = model2.predict(features_valid)


mse = mean_squared_error(target_valid, predicted_valid2)
print('EQM =', mse)
print('REQM =', mse ** 0.5)


- ##### Modelo de Regressão Linear simples

In [15]:
%%time
# Treinamento do modelo:
model = LinearRegression()
model.fit(features_train, target_train)

#Predições e análise:
predicted_valid = model.predict(features_valid)
mse = mean_squared_error(target_valid, predicted_valid)
print('EQM =', mse)
print('REQM =', mse ** 0.5)
print('R² =', model.score(features_valid, target_valid))

EQM = 9198078.191459762
REQM = 3032.8333603183282
R² = 0.5504613583211486
CPU times: user 13.2 s, sys: 3.89 s, total: 17.1 s
Wall time: 9.03 s


- ##### Modelo de Floresta aleatória com otimização de parâmetros:

In [16]:
%%time
#Treinamento do modelo com otimização do parâmetro max_depth:
for depth in range(1, 20, 5):
    model2 = RandomForestRegressor(n_estimators=10, max_depth=depth, random_state=12345)
    model2.fit(features_train, target_train)

# Predições e análise:
    predicted_valid = model2.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    print('Resultados para depth = ', depth)
    print('EQM =', mse)
    print('REQM =', mse ** 0.5)
    print('R² =', model2.score(features_valid, target_valid))

Resultados para depth =  1
EQM = 14434229.624293476
REQM = 3799.2406641713915
R² = 0.29455437930390704
Resultados para depth =  6
EQM = 5564804.175327047
REQM = 2358.9837166303305
R² = 0.7280307409750002
Resultados para depth =  11
EQM = 3983682.1992868045
REQM = 1995.9163808353305
R² = 0.8053050814016399
Resultados para depth =  16
EQM = 3405345.0980376904
REQM = 1845.3577154681122
R² = 0.8335702113033842
CPU times: user 1min 17s, sys: 925 ms, total: 1min 17s
Wall time: 1min 17s


In [17]:
%%time
#Treinamento do modelo com otimização do parâmetro n_estimators:
for est in range(1, 20, 5):
    model2 = RandomForestRegressor(n_estimators=est, max_depth=20, random_state=12345)
    model2.fit(features_train, target_train)
    predicted_valid = model2.predict(features_valid)
# Predições e análise:
    mse = mean_squared_error(target_valid, predicted_valid)
    print('Resultados para depth = ', est)
    print('EQM =', mse)
    print('REQM =', mse ** 0.5)
    print('R² =', model2.score(features_valid, target_valid))

Resultados para depth =  1
EQM = 4987381.15117836
REQM = 2233.244534568116
R² = 0.7562511970905945
Resultados para depth =  6
EQM = 3387013.425707953
REQM = 1840.384042994275
R² = 0.8344661370508368
Resultados para depth =  11
EQM = 3223813.652567026
REQM = 1795.4981627857562
R² = 0.8424422167071488
Resultados para depth =  16
EQM = 3165829.127705883
REQM = 1779.277698310717
R² = 0.8452761004817697
CPU times: user 2min 3s, sys: 806 ms, total: 2min 4s
Wall time: 2min 4s


- ##### Modelo LightGBM:

In [18]:
%%time
# Treinamento do modelo + otimização de parâmetros
train_data = lgb.Dataset(features_train, label=target_train)
valid_data = lgb.Dataset(features_valid, label=target_valid, reference=train_data)

for leaves in range(20, 100, 20):
    for rounds in range(100, 501, 200):
        
        param = {
            'objective': 'regression',  
            'metric': 'rmse',           
            'num_leaves': leaves,
            'learning_rate': 0.1,
            'feature_fraction': 0.9}
        
        num_round = rounds
        model_lgb = lgb.train(
            param, 
            train_data, 
            num_round, 
            valid_sets=[valid_data],
            callbacks=[lgb.early_stopping(10)]
        )

# Predições e análise dos dados
        predicted_valid_lgb = model_lgb.predict(features_valid)
        
        mse = mean_squared_error(target_valid, predicted_valid_lgb)
        print('Parâmetro num_leaves utilizado: ', leaves)
        print('Parâmetro num_round utilizado: ', rounds)
        print('LightGBM EQM =', mse)
        print('LightGBM REQM =', mse ** 0.5)
        print()


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.079924 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1225
[LightGBM] [Info] Number of data points in the train set: 265776, number of used features: 297
[LightGBM] [Info] Start training from score 4413.365319
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 1881.18
Parâmetro num_leaves utilizado:  20
Parâmetro num_round utilizado:  100
LightGBM EQM = 3538824.1176846563
LightGBM REQM = 1881.1762590689518

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006097 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1225
[LightGBM] [Info] Number of data points in

- ##### Modelo XGBoost:

In [19]:
%%time
#Definindo dados para treinamento e validação do modelo:
dtrain = xgb.DMatrix(features_train, label=target_train)
dtest = xgb.DMatrix(features_valid, label=target_valid)


params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}

# Treinamento do modelo
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

# Predições e análise
y_pred = model.predict(dtest)
predictions = [round(value) for value in y_pred]

mse = mean_squared_error(target_valid, y_pred)
print('LightGBM EQM =', mse)
print('LightGBM REQM =', mse ** 0.5)
print()

LightGBM EQM = 3603021.6798844724
LightGBM REQM = 1898.1627116463098

CPU times: user 4min 59s, sys: 549 ms, total: 5min
Wall time: 2min 30s


- ##### Modelo CatBoostRegressor:

In [20]:
%%time
#Treinando o modelo CatBoostRegressor e otimizando o parãmetro 'iterations'

for it in range(100, 1001, 100):
    model = CatBoostRegressor(iterations=it, random_seed=12345)
    model.fit(features_train, target_train, verbose=50)
    
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    print('Valor para parâmetro Iterations: ', it)
    print('CatBoost EQM =', mse)
    print('CatBoost REQM =', mse ** 0.5)
    print('CatBoost R² =', model.score(features_valid, target_valid))
    print()

Learning rate set to 0.5
0:	learn: 3283.0846091	total: 56.1ms	remaining: 5.55s
50:	learn: 1854.3777806	total: 1.37s	remaining: 1.32s
99:	learn: 1777.1592144	total: 2.67s	remaining: 0us
Valor para parâmetro Iterations:  100
CatBoost EQM = 3300062.593913563
CatBoost REQM = 1816.607440784487
CatBoost R² = 0.8387156941870209

Learning rate set to 0.366029
0:	learn: 3566.1824827	total: 15.4ms	remaining: 3.06s
50:	learn: 1881.3169645	total: 1.35s	remaining: 3.94s
100:	learn: 1797.3498052	total: 2.65s	remaining: 2.6s
150:	learn: 1752.1623273	total: 3.96s	remaining: 1.28s
199:	learn: 1718.7212299	total: 5.25s	remaining: 0us
Valor para parâmetro Iterations:  200
CatBoost EQM = 3147400.6267606914
CatBoost REQM = 1774.0914933454508
CatBoost R² = 0.846176758544317

Learning rate set to 0.263241
0:	learn: 3809.9540892	total: 32.4ms	remaining: 9.68s
50:	learn: 1917.0317663	total: 1.34s	remaining: 6.54s
100:	learn: 1835.8630909	total: 2.63s	remaining: 5.19s
150:	learn: 1786.7279311	total: 3.93s	remai

## Análise do modelo

No projeto, avaliamos a aplicação de 5 modelos diferentes de predição com base em dados. Podemos ressaltar que:
- Foi difícil a implementação desde o início devido a quantidade de dados presentes. Não apenas demorava a compilar os dados, como o próprio Kernel fechava e era necessário retomar o processo ajustando parâmetros, até que a primeira avaliação ajudou a encontrar a saída mais adequada (transformando apenas colunas categóricas em dummy, como deveria ter sido feito desde o início)
- Todos os modelos foram treinados considerando o modelo com colunas dummy.
- Tivemos altos valores de R² encontrados para os modelos que utilizam gradiente descendente. A diferença entre os modelos é a forma de aplicação no código e o tempo de execução, que ainda assim ficaram próximos.

Agora, utilizando os parâmetros otimizados para detectar o tempo de processamento de cada modelo para o melhor resultado:

In [21]:
%%time
# Modelo de Regressão Linear:
model = LinearRegression()
model.fit(features_train, target_train)

predicted_valid = model.predict(features_valid)
mse = mean_squared_error(target_valid, predicted_valid)
print('EQM =', mse)
print('REQM =', mse ** 0.5)
print('R² =', model.score(features_valid, target_valid))

EQM = 9198078.191459762
REQM = 3032.8333603183282
R² = 0.5504613583211486
CPU times: user 12.3 s, sys: 5.25 s, total: 17.6 s
Wall time: 9.23 s


In [22]:
%%time
# Modelo Floresta Aleatória:
model2 = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=12345)
model2.fit(features_train, target_train)

predicted_valid = model2.predict(features_valid)
mse = mean_squared_error(target_valid, predicted_valid)
print('EQM =', mse)
print('REQM =', mse ** 0.5)
print('R² =', model2.score(features_valid, target_valid))

EQM = 3242189.9191159992
REQM = 1800.6082081108036
R² = 0.8415441115017349
CPU times: user 35.9 s, sys: 296 ms, total: 36.2 s
Wall time: 36.1 s


In [23]:
%%time
# Modelo LightGBM:
train_data = lgb.Dataset(features_train, label=target_train)
valid_data = lgb.Dataset(features_valid, label=target_valid, reference=train_data)
param = {
    'objective': 'regression',  
    'metric': 'rmse',           
    'num_leaves': 80,
    'learning_rate': 0.1,
    'feature_fraction': 0.9}
        
num_round = 500
model_lgb = lgb.train(
    param, 
    train_data, 
    num_round, 
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(10)]
)

# Predições e análise dos dados
predicted_valid_lgb = model_lgb.predict(features_valid)       
mse = mean_squared_error(target_valid, predicted_valid_lgb)
print('LightGBM EQM =', mse)
print('LightGBM REQM =', mse ** 0.5)
print()


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006625 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1225
[LightGBM] [Info] Number of data points in the train set: 265776, number of used features: 297
[LightGBM] [Info] Start training from score 4413.365319
Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 1691.67
LightGBM EQM = 2861738.3738237363
LightGBM REQM = 1691.667335448591

CPU times: user 33.6 s, sys: 376 ms, total: 34 s
Wall time: 17.3 s


In [ ]:
%%time
# Modelo XGBoost:
dtrain = xgb.DMatrix(features_train, label=target_train)
dtest = xgb.DMatrix(features_valid, label=target_valid)
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'max_depth': 5,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8
}
num_rounds = 100
model = xgb.train(params, dtrain, num_rounds)

y_pred = model.predict(dtest)
predictions = [round(value) for value in y_pred]
mse = mean_squared_error(target_valid, y_pred)
print('XGBoost EQM =', mse)
print('XGBoost REQM =', mse ** 0.5)
print()

In [25]:
%%time
# Modelo CatBoostRegressor:
model = CatBoostRegressor(iterations=600, random_seed=12345)
model.fit(features_train, target_train, verbose=50)
    
predicted_valid = model.predict(features_valid)
mse = mean_squared_error(target_valid, predicted_valid)
print('CatBoost EQM =', mse)
print('CatBoost REQM =', mse ** 0.5)
print('CatBoost R² =', model.score(features_valid, target_valid))
print()

Learning rate set to 0.149836
0:	learn: 4100.2786074	total: 8.58ms	remaining: 5.14s
50:	learn: 2002.9483170	total: 1.29s	remaining: 13.9s
100:	learn: 1902.3334698	total: 2.67s	remaining: 13.2s
150:	learn: 1849.6291709	total: 3.96s	remaining: 11.8s
200:	learn: 1815.7782292	total: 5.27s	remaining: 10.5s
250:	learn: 1790.6454891	total: 6.57s	remaining: 9.13s
300:	learn: 1769.4618211	total: 7.93s	remaining: 7.88s
350:	learn: 1751.7647615	total: 9.24s	remaining: 6.56s
400:	learn: 1737.9645621	total: 10.5s	remaining: 5.23s
450:	learn: 1725.3369558	total: 11.8s	remaining: 3.91s
500:	learn: 1713.0298164	total: 13.1s	remaining: 2.6s
550:	learn: 1701.2163431	total: 14.4s	remaining: 1.28s
599:	learn: 1691.2373056	total: 15.7s	remaining: 0us
Valor para parâmetro Iterations:  1000
CatBoost EQM = 3077635.40289914
CatBoost REQM = 1754.3190710070787
CatBoost R² = 0.8495864016587082

CPU times: user 32.6 s, sys: 1.53 s, total: 34.1 s
Wall time: 17.1 s


### Resultados

- Com base nos dados observados, o modelo que melhor se adequou aos nossos dados foi o LightGBM, que obteve um REQM de 1691,67, pouco superior aos demais modelos porém com um tempo de processamento de 34s, muito diferente do XGBoost, por exemplo. O modelos CatBoost também se provou muito útil, com baixo tempo de processamento (também 34s) e com um REQM muito similar.
- Ficou claro a otimização dos modelos com o uso de Gradiente Descendente, que conseguem entregar resultados interessantes com baixo custo computacional.

# Checklist

Digite 'x' para verificar. Em seguida, pressione Shift + Enter.

- [x]  O Jupyter Notebook está aberto
- [ ]  O código está livre de erros
- [ ]  As células com o código foram organizadas em ordem de execução
- [ ]  Os dados foram baixados e preparados
- [ ]  Os modelos foram treinados
- [ ]  A análise de velocidade e qualidade dos modelos foi realizada